This tutorial can be downloaded [link](http://greatfire.uchicago.edu/west-public/West/raw/master/doc/tutorials/west_100.ipynb).

# 1.0 Getting Started: GW calculation

In order to compute the GW electronic structure of the silane molecule you need to run `pw.x`, `wstat.x` and `wfreq.x` in sequence. 

## Step 1: Ground State 

The ground state electronic structure of silane molecule with [QuantumEspresso](https://www.quantum-espresso.org/) is obtained by running `pw.x`. The pseudopotential files for **Si** and **H** in UPF format can be downloaded from: [QE-PP](https://www.quantum-espresso.org/pseudopotentials) database, or from [SG15](http://www.quantum-simulation.org/potentials/sg15_oncv/upf/) database. Check out the `pw.x` [input description](https://www.quantum-espresso.org/Doc/INPUT_PW.html) in order to generate an input file for QuantumEspresso called `pw.in`. 

Download these files in your current working directory:

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/silane/pw.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.0.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/Si_ONCV_PBE-1.1.upf

Let's inspect the ``pw.in`` file, input for ``pw.x``.

In [1]:
%%bash
cat pw.in

&control
calculation  = 'scf'
restart_mode = 'from_scratch'
pseudo_dir   = './'
outdir       = './'
prefix       = 'silane'
wf_collect   = .TRUE.
/
&system
ibrav           = 1
celldm(1)       = 20
nat             = 5
ntyp            = 2
ecutwfc         = 25.0
nbnd            = 10
assume_isolated ='mp'
/
&electrons
diago_full_acc = .TRUE.
/
ATOMIC_SPECIES
Si 28.0855  Si_ONCV_PBE-1.1.upf
H  1.00794   H_ONCV_PBE-1.0.upf
ATOMIC_POSITIONS bohr
Si      10.000000   10.000000  10.000000
H       11.614581   11.614581  11.614581
H        8.385418    8.385418  11.614581
H        8.385418   11.614581   8.385418
H       11.614581    8.385418   8.385418
K_POINTS {gamma}


Run ``pw.in`` on 2 cores.

In [ ]:
%%bash 
mpirun -n 2 pw.x -i pw.in > pw.out 

The output file ``pw.out`` contains information about the ground state calculation.

## Step 2: Screening

The static dielectric screening is computed using the projective dielectric eigendecomposition (PDEP) technique. Check out the ``wstat.x`` [input description](http://www.west-code.org/doc/West/latest/) and generate an input file for WEST called ``wstat.in``.

Download this file in your current working directory:

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/silane/wstat.in

Let's inspect the ``wstat.in`` file, input for ``wstat.x``.

In [2]:
%%bash
cat wstat.in

{
"input_west": {
"qe_prefix": "silane",
"west_prefix": "silane",
"outdir": "./"
},
"wstat_control": {
"wstat_calculation": "S",
"n_pdep_eigen": 50
}
}


Run ``wstat.in`` on 2 cores.

In [ ]:
%%bash 
mpirun -n 2 wstat.x -i wstat.in > wstat.out 

The output file ``wstat.out`` contains information about the PDEP iterations, and the dielectric eigenvalues can be found in the file ``west_prefix.wstat.save/wstat.json``.

## Step 3: GW

The GW electronic structure is computed treating the frequency integration of the correlation part of the self energy with the Contour Deformation techinique and by computing the dielectric screening at multipole frequencies with Lanczos iterations. Check out the ``wfreq.x`` [input description](http://www.west-code.org/doc/West/latest/) and generate an input file for WEST called ``wfreq.in``.

Download this file in your current working directory:

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/silane/wfreq.in

Let's inspect the ``wfreq.in`` file, input for ``wfreq.x``.

In [3]:
%%bash
cat wfreq.in

{
"input_west": {
"qe_prefix": "silane",
"west_prefix": "silane",
"outdir": "./"
},
"wstat_control": {
"wstat_calculation": "S",
"n_pdep_eigen": 50
},
"wfreq_control": {
"wfreq_calculation": "XWGQ",
"n_pdep_eigen_to_use": 50,
"qp_bandrange": [1,5],
"n_refreq": 300,
"ecut_refreq": 2.0
}
}


Run ``wstat.in`` on 2 cores.

In [ ]:
%%bash 
mpirun -n 2 wfreq.x -i wfreq.in > wfreq.out 

The output file ``wfreq.out`` contains information about the GW self-energy corrected electronic structure can be found in the file ``west_prefix.wstat.save/wfreq.json``.